In [1]:
import tensorflow as tf
import random
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import Adam, SGD, RMSprop


In [4]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize

# Define hyperparameter search space
param_grid = {
    "learning_rate": [0.001, 0.0005, 0.0001],
    "num_filters": [8, 16, 32],
    "optimizer": ["adam", "sgd", "rmsprop"],
    "dropout_rate": [0.2, 0.3, 0.5]
}

# Function to create CNN with given parameters
def create_cnn(learning_rate, num_filters, optimizer, dropout_rate):
    model = keras.Sequential([
        layers.Conv2D(num_filters, (3, 3), activation="relu", input_shape=(32, 32, 3)),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(num_filters*2, (3, 3), activation="relu"),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(10, activation="softmax")
    ])

    if optimizer == "adam":
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        opt = SGD(learning_rate=learning_rate)
    else:
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# Random Search
best_acc = 0
best_params = None

for _ in range(5):  # Run 5 random searches
    params = {key: random.choice(values) for key, values in param_grid.items()}
    print(f"Testing: {params}")

    model = create_cnn(**params)
    model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test), verbose=0)

    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"Accuracy: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_params = params

print(f"Best Accuracy: {best_acc:.4f}")
print(f"Best Parameters: {best_params}")

Testing: {'learning_rate': 0.0005, 'num_filters': 16, 'optimizer': 'sgd', 'dropout_rate': 0.2}
Accuracy: 0.1777
Testing: {'learning_rate': 0.0001, 'num_filters': 16, 'optimizer': 'rmsprop', 'dropout_rate': 0.5}
Accuracy: 0.4700
Testing: {'learning_rate': 0.001, 'num_filters': 8, 'optimizer': 'adam', 'dropout_rate': 0.5}
Accuracy: 0.5702
Testing: {'learning_rate': 0.001, 'num_filters': 16, 'optimizer': 'adam', 'dropout_rate': 0.2}
Accuracy: 0.6353
Testing: {'learning_rate': 0.0001, 'num_filters': 32, 'optimizer': 'adam', 'dropout_rate': 0.2}
Accuracy: 0.5560
Best Accuracy: 0.6353
Best Parameters: {'learning_rate': 0.001, 'num_filters': 16, 'optimizer': 'adam', 'dropout_rate': 0.2}


In [16]:
import tensorflow_datasets as tfds
import numpy as np
import tensorflow as tf
import random

# Load Tiny Shakespeare dataset (without as_supervised=True)
dataset = tfds.load("tiny_shakespeare", split='train')

# Extract text (Use a smaller subset for speed)
text = "".join([example["text"].numpy().decode('utf-8') for example in dataset])[:100000]  # Use only 100K characters

# Tokenization
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([text])

# Convert text to sequences
sequence = tokenizer.texts_to_sequences([text])[0]
vocab_size = len(tokenizer.word_index) + 1

# Prepare training sequences
seq_length = 40
subset_size = 5000  # Use only 5000 sequences for training
X, y = [], []

for i in range(subset_size):
    X.append(sequence[i:i+seq_length])
    y.append(sequence[i+seq_length])

X, y = np.array(X), np.array(y)

# Define hyperparameter search space
param_grid_rnn = {
    "learning_rate": [0.01, 0.005, 0.001],
    "num_units": [32, 64, 128],  # Increase to reasonable values
    "optimizer": ["adam", "rmsprop"],
    "dropout_rate": [0.2, 0.3]
}

# Function to create RNN model
def create_rnn(learning_rate, num_units, optimizer, dropout_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=64, input_length=seq_length),
        tf.keras.layers.SimpleRNN(num_units, return_sequences=True),
        tf.keras.layers.SimpleRNN(num_units),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(vocab_size, activation="softmax")
    ])

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate) if optimizer == "adam" else tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

    model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model

# Random Search for RNN (Reduced Iterations)
best_acc_rnn = 0
best_params_rnn = None

for _ in range(5):  # Reduce to 2 iterations for speed
    params_rnn = {key: random.choice(values) for key, values in param_grid_rnn.items()}
    print(f"Testing RNN with parameters: {params_rnn}")

    model_rnn = create_rnn(**params_rnn)
    model_rnn.fit(X, y, epochs=3, batch_size=64, verbose=0)  # Reduce batch size for faster training

    loss_rnn, acc_rnn = model_rnn.evaluate(X, y, verbose=0)
    print(f"RNN Accuracy: {acc_rnn:.4f}")

    if acc_rnn > best_acc_rnn:
        best_acc_rnn = acc_rnn
        best_params_rnn = params_rnn

print(f"\nBest RNN Accuracy: {best_acc_rnn:.4f}")
print(f"Best RNN Parameters: {best_params_rnn}")

Testing RNN with parameters: {'learning_rate': 0.005, 'num_units': 32, 'optimizer': 'adam', 'dropout_rate': 0.2}
RNN Accuracy: 0.3726
Testing RNN with parameters: {'learning_rate': 0.01, 'num_units': 32, 'optimizer': 'rmsprop', 'dropout_rate': 0.2}
RNN Accuracy: 0.2520
Testing RNN with parameters: {'learning_rate': 0.001, 'num_units': 32, 'optimizer': 'rmsprop', 'dropout_rate': 0.3}
RNN Accuracy: 0.2426
Testing RNN with parameters: {'learning_rate': 0.01, 'num_units': 64, 'optimizer': 'rmsprop', 'dropout_rate': 0.3}
RNN Accuracy: 0.1346
Testing RNN with parameters: {'learning_rate': 0.001, 'num_units': 128, 'optimizer': 'rmsprop', 'dropout_rate': 0.2}
RNN Accuracy: 0.2860

Best RNN Accuracy: 0.3726
Best RNN Parameters: {'learning_rate': 0.005, 'num_units': 32, 'optimizer': 'adam', 'dropout_rate': 0.2}
